In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, f1_score, accuracy_score, silhouette_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from LSTM_model import LSTM

%matplotlib inline

In [2]:
# Import data

edgelist = pd.read_csv('../data/elliptic_txs_edgelist.csv')
data = pd.read_csv('../data/merged_df.csv')

In [3]:
data['class'] = data['class'].map({'licit': 0, 'illicit': 1, 'unknown': 'unknown'})
data = data[data['class'] != 'unknown']

In [7]:
# Create train and test sets

X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

# Create train and test sets for autoencoder
y_train_lstm = X_train['class']
y_test_lstm = X_test['class']

X_train_lstm = X_train.drop(['txId', 'Time step', 'class'], axis=1)
X_test_lstm = X_test.drop(['txId', 'Time step', 'class'], axis=1)

# Convert to tensors

X_train_lstm = torch.tensor(X_train_lstm.values).float()
X_test_lstm = torch.tensor(X_test_lstm.values).float()

y_train_lstm = y_train_lstm.astype('int')
y_test_lstm = y_test_lstm.astype('int')

y_train_lstm = torch.tensor(y_train_lstm.values).long()
y_test_lstm = torch.tensor(y_test_lstm.values).long()

In [14]:
# Load a lstm model from a pt file
lstm = LSTM(input_dim=X_train_lstm.size(-1), hidden_dim=50, output_dim=1, num_layers=2)

In [15]:
lstm.load_state_dict(torch.load('../models/lstm_model.pt'))

<All keys matched successfully>

In [21]:
lstm.eval()
with torch.no_grad():
    outputs = lstm(X_test_lstm)
    predicted = (torch.sigmoid(outputs) > 0.5).float()
    total = len(y_test_lstm) 
    correct = (predicted.view(-1) == torch.Tensor(y_test_lstm).float()).sum().item()
    print('Test Accuracy: %f %%' % (100 * correct / total))

Test Accuracy: 97.916890 %
